## Critical path showcase



This notebook provides examples of a simulation that takes a number of barges and a total amount to be moved. We show the critical path (as a pd.DataFrame and a gantt plot) for a number of different simulation (data) and methods.

For this example we work with the following methods to determine critical path:
* __DependenciesFromRecordedActivities__
* __DependenciesFromSimpy__

The latter method is recommended as the dependencies found with `from simpy` correspond wth the events as scheduled within the simulation, whereas `from recorded activities` assumes dependencies from recorded data (dependencies assumed in case of identical timestamps and simulation objects).

For this example we work with the following simulations to determine critical path:
* two barges move an amount from site 0 to site 1. When the container on the site 1 reaches a certain level, another (third) vessel may move the amount from site 1 to site 2.
* four barges move an amount from site 0 to site 1. When the container on the site 1 reaches a certain level, another (fifth) vessel moves the entire amount from site 1 to site 2.

Note that the barges have a start event (delay) and velocity based on their number (i.e. barge 0 has zero delay, barge1 100seconds, barge2 200 seconds et cetera).


#### 0. Import libraries

In [1]:
import simpy
import shapely

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

from openclsim.critical_path.dependencies_from_simpy_step import DependenciesFromSimpy, AlteredStepEnv
from openclsim.critical_path.dependencies_from_recorded_activities import DependenciesFromRecordedActivities


#### 1. Use a function definition to get some simulation data

In this notebook we can play around with the nr of barges and total amount to be moved. After calling this function with desired input parameters we can investigate the data of the simulations and inspect the gantt chart.

In [2]:
def demo_data(nr_barges, total_amount, env=None):
    """
    Run a simulation where <nr_barges> barges need to shift an amount of <total_amount>
    from site 1 (nr resources=1) to site 2 (nr resources=4) whereafter another vessel can come into action.

    Parameters
    ----------
    nr_barges : int
        Number of barges in the simulation.
    total_amount : int
        Total amount to be transported in the simulation.
    env : simpy.Environment or class that inherits from simpy.Environment
        Optional. If None, default to simpy Environment
    """
    Site = type(
        "Site",
        (
            core.Identifiable,
            core.Log,
            core.Locatable,
            core.HasContainer,
            core.HasResource,
        ),
        {},
    )
    TransportProcessingResource = type(
        "TransportProcessingResource",
        (
            core.Identifiable,
            core.Log,
            core.ContainerDependentMovable,
            core.Processor,
            core.HasResource,
        ),
        {},
    )

    simulation_start = 0
    if env is None:
        my_env = simpy.Environment(initial_time=simulation_start)
    else:
        my_env = env(initial_time=simulation_start)

    registry = {}

    location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)

    data_from_site = {
        "env": my_env,
        "name": "from_site",
        "geometry": location_from_site,
        "capacity": 4 * total_amount,
        "level": 4 * total_amount,
        "nr_resources": 1,
    }
    from_site = Site(**data_from_site)

    location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)
    data_to_site = {
        "env": my_env,
        "name": "to_site",
        "geometry": location_to_site,
        "capacity": 3*total_amount,
        "level": 0,
        "nr_resources": 4,
    }
    to_site = Site(**data_to_site)

    location_to_site2 = shapely.geometry.Point(4.35222222, 52.11428333)
    data_to_site2 = {
        "env": my_env,
        "name": "to_site2",
        "geometry": location_to_site2,
        "capacity": total_amount,
        "level": 0,
        "nr_resources": 4,
    }
    to_site2 = Site(**data_to_site2)

    vessels = {}

    for i in range(nr_barges):
        vessels[f"vessel{i}"] = TransportProcessingResource(
            env=my_env,
            name=f"barge_{i}",
            geometry=location_from_site,
            capacity=10,
            compute_v=lambda x: 10,
        )

    # vessel_last wait till whiletask done
    vessel_last = TransportProcessingResource(
        env=my_env,
        name="vessel_last",
        geometry=location_from_site,
        capacity=10,
        compute_v=lambda x: 10,
    )
    vessels["vessel_last"] = vessel_last

    activities = {}
    for i in range(nr_barges):
        amount = 5  # handle loading
        duration = 2000  # sailing and unloading

        requested_resources = {}
        activities[f"activity{i}"] = model.WhileActivity(
            env=my_env,
            name=f"while_sequential_activity_subcycle{i}",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name=f"sequential_activity_subcycle{i}",
                    registry=registry,
                    sub_processes=[
                        model.BasicActivity(
                            env=my_env,
                            name="basic activity:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            duration=duration,
                            additional_logs=[vessels[f"vessel{i}"]],
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing empty:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            mover=vessels[f"vessel{i}"],
                            destination=from_site,
                            duration=duration - 200*i
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="loading:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            processor=vessels[f"vessel{i}"],
                            origin=from_site,
                            destination=vessels[f"vessel{i}"],
                            amount=amount,
                            duration=500 * amount,
                            requested_resources=requested_resources,
                            start_event=[
                            {
                                "type": "container",
                                "concept": from_site,
                                "state": "gt",
                                "level": amount
                            }
                            ]
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing full:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            mover=vessels[f"vessel{i}"],
                            destination=to_site,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="unloading:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            processor=vessels[f"vessel{i}"],
                            origin=vessels[f"vessel{i}"],
                            destination=to_site,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                    ],
                )
            ],
            condition_event=[
                {
                    "type": "container",
                    "concept": from_site,
                    "state": "lt",
                    "level": 2.8*total_amount
                }
            ],
              start_event={
                "type":"time",
                "start_time":i*100,
            }
        )

    # now add activity for vessel last, once v1 and v2 are done
    requested_resources = {}
    amount = 5
    duration = 2000
    activities[f"activity_last_vessel"] = model.WhileActivity(
            env=my_env,
            name=f"while_last_vessel",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name="sequential_last_vessel",
                    registry=registry,
                    sub_processes=[
                        model.BasicActivity(
                            env=my_env,
                            name="basic activity vessel_last",
                            registry=registry,
                            duration=duration,
                            additional_logs=[vessel_last]
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing empty: vessel_last",
                            registry=registry,
                            mover=vessel_last,
                            destination=to_site,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="loading vessel_last",
                            registry=registry,
                            processor=vessel_last,
                            origin=to_site,
                            destination=vessel_last,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing full vessel_last",
                            registry=registry,
                            mover=vessel_last,
                            destination=to_site2,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="unloading vessel_last",
                            registry=registry,
                            processor=vessel_last,
                            origin=vessel_last,
                            destination=to_site2,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                    ],
                )],
            start_event=[
                {
                    "type": "container",
                     "concept": to_site,
                     "state": "gt",
                     "level": 30
                }
            ],
            condition_event=[
                {
                    "type": "container",
                    "concept": to_site2,
                    "state": "full",
                }
            ],)

    model.register_processes(list(activities.values()))
    my_env.run()

    return {
        "env": my_env,
        "object_list": [from_site, to_site, to_site2] + list(vessels.values()),
        "activity_list": list(activities.values()),
    }



#### 2. Get critical path with simpy

* _method: DependenciesFromSimpy_
* _simulation: simulation with 2 barges_

First we run the simulation, then we initialize the Dependencies instance in order to get the recorded activities (with flag 'is_critical') and show these in a Gantt plot.

In [3]:
# Run the simulation.
simulation_data_out = demo_data(2, 100, env=AlteredStepEnv)

In [4]:
# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_simpy = DependenciesFromSimpy(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,d83e0f42-d83c-49e0-b979-78a5e615b85b,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,a420e414-448b-47e0-8c7c-73cb0613b211,False
9,17c90694-1926-4ad0-a121-4fcd9a87c369,basic activity:barge_1,barge_1,1970-01-01 00:01:40,1970-01-01 00:35:00,0 days 00:33:20,ACTIVE,c871b460-78ec-47e1-8d07-3b52f8c4d510,True
15,0b300bb8-22c2-4314-8804-d22ac9aef1e3,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,92a316b6-0507-4cb6-b3cc-666fafca06fa,False
18,0493c740-0109-45e1-b358-1690e35e9952,sailing empty:barge_1,barge_1,1970-01-01 00:35:00,1970-01-01 01:05:00,0 days 00:30:00,ACTIVE,8201acc8-fc66-49e7-beb3-cd7a6df0046e,True
21,89f14b1b-e39b-4850-8d2d-1c586225e35e,loading:barge_1,barge_1,1970-01-01 01:05:00,1970-01-01 01:46:40,0 days 00:41:40,ACTIVE,8a6060d4-bf6d-426b-8464-5d250b267cbb,True


In [5]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.make_plotly_gantt_chart()

#### 3. Get critical path with recorded activities

* _method: DependenciesFromRecordedActivities_
* _simulation: simulation with 2 barges_

First we run the simulation, then we get the recorded activities (with flag 'is_critical') and show these in a gantt plot.

In [6]:
# Run the simulation.
simulation_data_out = demo_data(2, 100)

# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_activities = DependenciesFromRecordedActivities(**simulation_data_out)
critical_df = my_cp_dependencies_from_activities.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,05c83efd-6a3c-4b37-a88f-6b66f27416e6,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,5e3ac05a-981f-49f7-a885-3711d9e80ba0,False
9,3a2975a3-9736-426f-b8cb-9055e8a3d32a,basic activity:barge_1,barge_1,1970-01-01 00:01:40,1970-01-01 00:35:00,0 days 00:33:20,ACTIVE,3e8ff897-62eb-4fd1-98d8-6253775ff66e,False
15,b846ccfa-30a7-4f49-809b-ebdc376be673,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,a7fda34f-6f51-4789-9014-7851bac6958c,False
18,eb17898a-f938-4ef3-9186-6ab09eb4ce0a,sailing empty:barge_1,barge_1,1970-01-01 00:35:00,1970-01-01 01:05:00,0 days 00:30:00,ACTIVE,9fe59eca-0992-44c5-b718-dbdc3402b100,False
21,2f4f8a92-da98-4c17-93d4-395923ab9d2f,loading:barge_1,barge_1,1970-01-01 01:05:00,1970-01-01 01:46:40,0 days 00:41:40,ACTIVE,89551150-253a-4006-a163-b239a6b5ecc1,False


In [7]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.make_plotly_gantt_chart()

#### 4. Get critical path with simpy

* _method: DependenciesFromSimpy_
* _simulation: simulation with 4 barges_

First we run the simulation, then we initialize the Dependencies instance in order to get the recorded activities (with flag 'is_critical') and show these in a Gantt plot.

In [8]:
# Run the simulation.
simulation_data_out = demo_data(4, 100, env=AlteredStepEnv)

# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_simpy = DependenciesFromSimpy(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,b476859a-04fd-4508-98db-72845220ab1b,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,ab4ce31f-0c0a-4a68-a07e-d455c1fc5a78,False
11,3aea7b5b-b612-4fec-9723-e2831a5a5efc,basic activity:barge_1,barge_1,1970-01-01 00:01:40,1970-01-01 00:35:00,0 days 00:33:20,ACTIVE,08210675-3602-4a39-9a51-77174c135d6d,False
17,cd6efb31-f6c8-4e91-b3b8-605362b46ef7,basic activity:barge_2,barge_2,1970-01-01 00:03:20,1970-01-01 00:36:40,0 days 00:33:20,ACTIVE,e470e680-e89f-43f7-b5b8-b3adde9bb76d,False
23,0793c61b-d497-4384-9a67-5bd41972fa75,basic activity:barge_3,barge_3,1970-01-01 00:05:00,1970-01-01 00:38:20,0 days 00:33:20,ACTIVE,dab58f27-a87a-412e-8fe9-56cadf270534,True
29,12d3e4ba-270d-4571-bff9-88951db51945,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,de155d0d-67d5-464a-86e3-63d8db5b8dfd,False


In [9]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.make_plotly_gantt_chart()